# LangGraph Email Processing System

In [2]:
%pip install -U langchain-ollama langgraph

  Using cached msgpack-1.1.0-cp311-cp311-win_amd64.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
   -------- ------------------------------ 30.7/134.8 kB 660.6 kB/s eta 0:00:01
   --------------------------------- ------ 112.6/134.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 134.8/134.8 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
   ---------------------------------------- 41.2/41.2 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.0 kB ? eta -:--:--
   ---------------------------------------- 46.0/46.0 kB 1.2 MB/s eta 0:00:00
Using cached msgpack-1.1.0-cp311-cp311-win_amd64.whl (74 kB)
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.4
    Uninstalling msgpack-1.0.4:
      Successfully uninstalled msgpack-1.0.4
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip list

Package                                  Version
---------------------------------------- ---------------
accelerate                               1.4.0
agate                                    1.9.1
aiofiles                                 23.2.1
aiohappyeyeballs                         2.4.4
aiohttp                                  3.11.11
aiosignal                                1.3.2
aiosqlite                                0.20.0
alembic                                  1.14.1
altair                                   5.5.0
annotated-types                          0.7.0
anyio                                    4.4.0
apache-airflow                           2.10.4
apache-airflow-providers-common-compat   1.3.0
apache-airflow-providers-common-io       1.5.0
apache-airflow-providers-common-sql      1.21.0
apache-airflow-providers-fab             1.5.2
apache-airflow-providers-ftp             3.12.0
apache-airflow-providers-http            5.0.0
apache-airflow-providers-imap           


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


![image](images/langgraph_email_processing.png)

In [1]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, END, START
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage

In [2]:
class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    draft_response: Optional[str]
    messages: List[Dict[str, Any]]
    email_category: Optional[str]
    spam_reason: Optional[str]

In [9]:
model = ChatOllama(model="llama3.2:3b")

def read_email(state: EmailState):
    """Alfred reads and logs the incomming email"""
    email = state["email"]

    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")

    return {}

def classify_email(state: EmailState):
    """Alfred uses an LLM to determine if the email is spam or legitimate"""
    email = state["email"]
    
    prompt = f"""
    As Alfred the butler, analyze this email and determine if it is spam or legitimate.

    Email:
    From: {email["sender"]}
    Subject: {email["subject"]}
    Body: {email["body"]}

    First, determine if this email is spam. If it is spam, explain why, MUST include "spam reason:" text.
    If it is legitimate, DO NOT include any "spam" word in response, do not include "reason" or "spam reason", then categorize it as "inquiry", "complaint", "thank you", "request", or "information".
    """

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    print(response)

    response_text = response.content.lower()
    is_spam = "spam reason" in response_text

    spam_reason = None
    if is_spam and "reason:" in response_text:
        spam_reason = response_text.split("reason:")[1].strip()

    email_category = "general"
    if not is_spam:
        categories = ["inquiry", "complaint", "thank you", "request", "information"]
        for category in categories:
            if category in response_text:
                email_category = category
                break

    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt}
        , {"role": "assistant", "content": response.content}
    ]

    return {
        "is_spam": is_spam
        , "spam_reason": spam_reason
        , "email_category": email_category
        , "messages": new_messages
    }

def handle_spam(state: EmailState):
    """Alfred discards spam email with a note"""
    print(f"Alfred has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email as been moved to the spam folder.")

    return {}

def draft_email(state: EmailState):
    """Alfred drafts a preliminary response for legitimate emails"""
    email = state["email"]
    category = state["email_category"] or "general"

    prompt = f"""
    As Alfred the butler, draft a polite preliminary response to this email.
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    
    This email has been categorized as: {category}
    
    Draft a brief, professional response that Mr. Hugg can review and personalize before sending.
    """

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]

    return {
        "draft_response": response.content,
        "messages": new_messages
    }

def notify_mr_hugg(state: EmailState):
    """Alfred notifies Mr. Hugg about the email and presents the draft response"""
    email = state["email"]
    
    print("\n" + "="*50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"Category: {state['email_category']}")
    print("\nI've prepared a draft response for your review:")
    print("-"*50)
    print(state["draft_response"])
    print("="*50 + "\n")
    
    # We're done processing this email
    return {}
    

In [10]:
def route_email(state: EmailState) -> str:
    """Determine the next step based on spam classification"""
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"

In [11]:
email_graph = StateGraph(EmailState)

email_graph.add_node("read_email", read_email)
email_graph.add_node("classify_email", classify_email)
email_graph.add_node("handle_spam", handle_spam)
email_graph.add_node("draft_email", draft_email)
email_graph.add_node("notify_mr_hugg", notify_mr_hugg)

email_graph.add_edge(START, "read_email")
email_graph.add_edge("read_email", "classify_email")

email_graph.add_conditional_edges(
    "classify_email"
    , route_email
    , {
        "spam": "handle_spam"
        , "legitimate": "draft_email"
    }
)

email_graph.add_edge("handle_spam", END)
email_graph.add_edge("draft_email", "notify_mr_hugg")
email_graph.add_edge("notify_mr_hugg", END)

compiled_graph = email_graph.compile()

## Run the app

In [12]:
legitimate_email = {
    "sender": "john.smith@example.com",
    "subject": "Question about your services",
    "body": "Dear Mr. Hugg, I was referred to you by a colleague and I'm interested in learning more about your consulting services. Could we schedule a call next week? Best regards, John Smith"
}

spam_email = {
    "sender": "winner@lottery-intl.com",
    "subject": "YOU HAVE WON $5,000,000!!!",
    "body": "CONGRATULATIONS! You have been selected as the winner of our international lottery! To claim your $5,000,000 prize, please send us your bank details and a processing fee of $100."
}

In [13]:
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke({
    "email": legitimate_email
    , "is_spam": None
    , "spam_reason": None
    , "email_category": None
    , "draft_response": None
    , "messages": []
})


Processing legitimate email...
Alfred is processing an email from john.smith@example.com with subject: Question about your services
content='After analyzing the email, I would conclude that it is a legitimate inquiry. Here\'s my reasoning:\n\nThe email appears to be from a real person, John Smith, who claims to have been referred by a colleague. The email does not contain any suspicious links, attachments, or overly promotional language that is typically found in spam emails.\n\nFurthermore, the tone of the email is polite and professional, suggesting that Mr. Hugg is a reputable service provider. The request for a call to discuss consulting services further supports this assessment, as it implies a genuine interest in learning more about Mr. Hugg\'s services.\n\nI would categorize this email as an "inquiry" since it represents a potential lead for Mr. Hugg to provide his consulting services to John Smith.' additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': 

In [14]:
print("\nProcessing legitimate email...")
spam_result = compiled_graph.invoke({
    "email": spam_email
    , "is_spam": None
    , "spam_reason": None
    , "email_category": None
    , "draft_response": None
    , "messages": []
})


Processing legitimate email...
Alfred is processing an email from winner@lottery-intl.com with subject: YOU HAVE WON $5,000,000!!!
content='An intriguing email, sir. Upon analyzing the content, I have determined that this email is likely spam.\n\nSpam reason: The email contains an over-the-top and sensational subject line, which is a common tactic used by scammers to lure victims into responding. Additionally, the email asks for sensitive information (bank details) without providing any credible evidence or verification of the winner\'s identity. This is a classic characteristic of phishing attempts.\n\nHowever, if I were to categorize this email based on its intended purpose, I would say it falls under "inquiry".' additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-03-20T02:39:43.8897314Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7693687500, 'load_duration': 18825100, 'prompt_eval_count': 196, 'prompt_eval_duration': 1918000000, 'eval_co